# **METÓDOS DAS DIFERENÇAS FINITAS NEWMAN** 

PROBLEMAS DE CONTORNO


In [145]:
from sympy import symbols, IndexedBase, Sum, Eqn, simplify
import numpy as np
from algebra_with_sympy import *


In [146]:
i, h, z = symbols('i h z')
y = IndexedBase('y')
x = IndexedBase('x')

yll = Eqn(y[i] ,(y[i+1]-2*y[i]+y[i-1])/(h**2)) 
ylprog = Eqn(y[i],((y[i+1]-y[i])/h))
ylregr = Eqn(y[i],((y[i]-y[i-1])/h))
ylcent = Eqn(y[i],((y[i+1]-y[i-1])/(2*h)))

display(yll, ylprog, ylregr, ylcent)


Equation(y[i], (y[i + 1] + y[i - 1] - 2*y[i])/h**2)

Equation(y[i], (y[i + 1] - y[i])/h)

Equation(y[i], (-y[i - 1] + y[i])/h)

Equation(y[i], (y[i + 1] - y[i - 1])/(2*h))

**Discretizando**

In [147]:
edo = Eqn(-yll.rhs + y[i], x[i])  #pegando o lado direito da equação yll e aplicando na função
edo = simplify(edo)*h**2
# Agrupando os elementos em comum
common_y_terms = edo.lhs.collect(y[i])
remaining_terms = edo.rhs

# Equação discretizada final
grouped_equation = Eq(common_y_terms, remaining_terms)
grouped_equation

Eq((h**2 + 2)*y[i] - y[i + 1] - y[i - 1], h**2*x[i])

# Condição de Robin
Tenho aqui o meu vetor com h=0.25
| i |  x  | y |
| --- | --- | --- |
| 0 | 0.00 | -1  |
| 1 | 0.25 |     |
| 2 | 0.50 |     |
| 3 | 0.75 |     |
| 4 | 1.00 |     |
| 5 | 1.25 |     |

E, dado:
y'(1.25) = 1
y (0) = -1

In [148]:
#Meus vetores
y1 = 1
xv  = [0, 0.25, 0.50, 0.75, 1, 1.25]
yv  = [-1, nan, nan, nan, nan,nan]

In [149]:
n=len(xv)-1
eq = {}
for k in range(1,n):
    equation = grouped_equation.subs(i, k)
    eq[k] = equation
    display(eq[k])

    


Eq((h**2 + 2)*y[1] - y[0] - y[2], h**2*x[1])

Eq((h**2 + 2)*y[2] - y[1] - y[3], h**2*x[2])

Eq((h**2 + 2)*y[3] - y[2] - y[4], h**2*x[3])

Eq((h**2 + 2)*y[4] - y[3] - y[5], h**2*x[4])

Substituindo o valor de y{0} e y{n}:

In [150]:

z = yv[0] #valor inicial y0 que temos
for k in range(1, len(eq)+1):
    eq[k] = eq[k].evalf(4, subs={y[0]: z})
    

#Colocando o valor y0 para o outro lado da equação
#Neste caso só preciso mudar a primeira Equação
common_y_terms = eq[1].lhs.collect(y[i])
remaining_terms = eq[1].rhs
eq[1] = Eq(common_y_terms + z, remaining_terms+z) #aqui estou jogando o valor de y0 para o outro lado


#Mudando a ultima equação
n = len(eq)+1
ylregr = ylregr.subs(i, n)
ylregr_lhs = ylregr.lhs.subs(y[n], y1)
equation_moved = Eq(ylregr_lhs * h, ylregr.rhs * h)
eq_moved = Eq(equation_moved.rhs + y[n-1],equation_moved.lhs+y[n-1])
#substituindo na ultima equacao
eqmoved = Eq(eq[n-1].lhs - eq_moved.rhs + eq_moved.lhs, eq[n-1].rhs)
# Agrupando os elementos em comum
common_y_terms = eqmoved.lhs.collect(y[n-1])
remaining_terms = eqmoved.rhs

# Equação discretizada final
grouped_equation = Eq(common_y_terms, remaining_terms)
const = -1
eq[n-1] = Eq(grouped_equation.lhs  - const*h, grouped_equation.rhs + h) 

for k in range(1, len(eq)+1):
    display(eq[k])

Eq((h**2 + 2.0)*y[1] - y[2], h**2*x[1] - 1)

Eq((h**2 + 2.0)*y[2] - y[1] - y[3], h**2*x[2])

Eq((h**2 + 2.0)*y[3] - y[2] - y[4], h**2*x[3])

Eq((h**2 + 1.0)*y[4] - y[3], h**2*x[4] + h)

In [151]:
matrix = [[0 for _ in range(n-1)] for _ in range(n-1)]
matrix
for i in range(1,n):
   # print("Eq ", i, "---------------")
    for j in range(len(eq[i].args[0].args)): #len(eq[2].args[0].args) é a qtd de y que tenho
        if eq[i].args[0].args[j].args[1].args[1] == 1:
            matrix[i-1][0] = eq[i].args[0].args[j].args[0]
        if eq[i].args[0].args[j].args[1].args[1] == 2:
            matrix[i-1][1] = eq[i].args[0].args[j].args[0]
        if eq[i].args[0].args[j].args[1].args[1] == 3:
            matrix[i-1][2] = eq[i].args[0].args[j].args[0]
        if eq[i].args[0].args[j].args[1].args[1] == 4:
            matrix[i-1][3] = eq[i].args[0].args[j].args[0]
left = []
for i in range(1,n):
    left.append(eq[i].rhs)
    

display(Matrix(matrix))

Matrix([
[h**2 + 2.0,         -1,          0,          0],
[        -1, h**2 + 2.0,         -1,          0],
[         0,         -1, h**2 + 2.0,         -1],
[         0,          0,         -1, h**2 + 1.0]])

In [152]:
A = np.array(matrix.copy())
for i in range(len(matrix)):
    A[i, i] = A[i, i].subs(h, 0.25)

B = np.array(left.copy())
for i in range(len(matrix)):
    B[i] = B[i].subs(h, 0.25)
    B[i] = B[i].subs(x[i+1],xv[i+1])
    
matrix_aumentada = np.concatenate((Matrix(A),Matrix(B)), axis=1)

print("Meu Sistema ROBIN:")
display(Matrix(matrix))
display(Matrix(left))
    
print("Meu Sistema:")
display(Matrix(A))
display(Matrix(B))

Meu Sistema ROBIN:


Matrix([
[h**2 + 2.0,         -1,          0,          0],
[        -1, h**2 + 2.0,         -1,          0],
[         0,         -1, h**2 + 2.0,         -1],
[         0,          0,         -1, h**2 + 1.0]])

Matrix([
[h**2*x[1] - 1],
[    h**2*x[2]],
[    h**2*x[3]],
[h**2*x[4] + h]])

Meu Sistema:


Matrix([
[2.0625,     -1,      0,      0],
[    -1, 2.0625,     -1,      0],
[     0,     -1, 2.0625,     -1],
[     0,      0,     -1, 1.0625]])

Matrix([
[-0.984375],
[  0.03125],
[ 0.046875],
[   0.3125]])

In [153]:
from thomas_system_solve import thomas
print("SOLUÇÃO ROBIN")
thomas(matrix_aumentada)


SOLUÇÃO ROBIN
Solução
x 1 = -0.4379
x 2 = 0.0812
x 3 = 0.0656
x 4 = 0.0
